# protein prediction appendix fig

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from zebrafish_ms2_paper.trace_analysis import extract_traces,predict_protein, predict_protein_v3
from zebrafish_ms2_paper.utils import pboc_rc, style_axes, colors, fontsize
from matplotlib import rc, rcParams
import pandas as pd
import pickle
import matplotlib as mpl


In [2]:
%matplotlib qt

In [10]:
colors

{'green': '#7AA974',
 'light_green': '#BFD598',
 'pale_green': '#DCECCB',
 'yellow': '#EAC264',
 'light_yellow': '#F3DAA9',
 'pale_yellow': '#FFEDCE',
 'blue': '#738FC1',
 'light_blue': '#A9BFE3',
 'pale_blue': '#C9D7EE',
 'red': '#D56C55',
 'light_red': '#E8B19D',
 'pale_red': '#F1D4C9',
 'purple': '#AB85AC',
 'light_purple': '#D4C2D9',
 'dark_green': '#7E9D90',
 'dark_brown': '#905426'}

In [3]:
fontsize = 8
linewidth = 2
markersize = 8

In [4]:
"""specify the path to the folder of Supplemental Data Files"""
# REPLACE THIS WITH YOUR PATH
datadir = r'/home/brandon/Downloads/SupplementalDataFiles'

# define paths to particular files
path_to_spots = datadir + '/Supplemental_Data_File_1-Spots.pkl'
path_to_non_blank_timepoints = datadir + '/Supplemental_Data_File_3-Non-Blank_Time_Points.pkl'


In [5]:
rcParams.update(pboc_rc)
rcParams['pdf.fonttype'] = 42

In [6]:
# correct time stamps
with open(path_to_non_blank_timepoints,'rb') as fp:
    non_blank_timepoints = pickle.load(fp)

In [7]:
plt.close('all')

## Fig5D
Plot an example of a binarized trace

In [9]:
# load the manual dorado dataset
df = pd.read_pickle(path_to_spots)

good_ids = [2715]

df = df[[nuc in good_ids for nuc in df.nucleus_id]]

# extract culled traces
traces = extract_traces(df, method='gauss3d_dog')

            

/home/brandon/Documents/Code/zebrafish-ms2-paper/zebrafish_ms2_paper/trace_analysis.py:42: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  inten_arr[i] = sub_sub_df.get(method)


In [32]:
t_arr, inten_arr, nucleus = traces[0]
t_arr = non_blank_timepoints[t_arr.astype('int')]

plt.figure(figsize=(6, 2.5))
plt.plot(t_arr, inten_arr / np.max(inten_arr), linewidth=linewidth, label='$her1$-MS2 (experimental data)', color=colors['green'])

# start from protein = 0:
Tmax = np.max(t_arr)
t_eval = np.arange(np.min(t_arr), np.max(t_arr))
initial_condition = (0.0, 0.0, 0.0)
mrna, protein, fp = predict_protein_v3(inten_arr, t_arr, Tmax=Tmax, t_eval=t_eval, initial_conditions=initial_condition)

plt.plot(t_eval, protein / np.max(protein), linewidth=linewidth, color=colors['purple'], label='predicted Her1 protein, low initial protein')

# start from protein = 1:
Tmax = np.max(t_arr)
t_eval = np.arange(np.min(t_arr), np.max(t_arr))
initial_condition = (0.0, 1e8, 0.0)
mrna, protein, fp = predict_protein_v3(inten_arr, t_arr, Tmax=Tmax, t_eval=t_eval, initial_conditions=initial_condition)

plt.plot(t_eval, protein / np.max(protein), '--', linewidth=linewidth, color=colors['red'], label='predicted Her1 protein, high initial protein')

plt.xlabel('time (min)', fontsize=fontsize)
plt.ylabel('$her1$-MS2 fluorescence intensity, \npredicted Her1 protein \n(fraction of max)', fontsize=fontsize)
plt.xlim([0, 75])
plt.ylim([0, 1.6])
ax = style_axes(plt.gca(), fontsize=fontsize)
plt.legend(fontsize=fontsize, facecolor='w')
plt.tight_layout()

/tmp/ipykernel_4102595/628345005.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(6, 2.5))


In [33]:
plt.savefig(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/figures/appendix_protein_prediction_fig.pdf')